In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-llama2-13b-32k-instructions')

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-llama2-13b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [5]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [6]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nkwsp tu apa [/INST] Kumpulan Wang Simpanan Pekerja (KWSP) merupakan sebuah badan berkanun yang ditubuhkan di bawah Akta Kumpulan Wang Simpanan Pekerja 1991 untuk menguruskan simpanan persaraan pekerja dan melaksanakan program persaraan kepada pekerja dan tanggungan mereka. Visi KWSP adalah untuk menjadi pihak berkuasa persaraan terkemuka dalam membina dan membangunkan simpanan untuk masa depan yang lebih baik. </s>'

In [7]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [8]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

<s> [INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun [/INST] Jabatan Koko Malaysia ialah agensi kerajaan yang bertanggungjawab untuk pembangunan dan pengawalseliaan industri koko Malaysia. Sementara itu, pengeluaran koko adalah sebahagian kecil daripada ekonomi Malaysia. Antara faktor yang menyumbang kepada ini ialah kos tenaga buruh yang tinggi, kekurangan tanah untuk penanaman koko, dan kekurangan penglibatan dalam sektor hiliran. Walau bagaimanapun, industri koko Malaysia mempunyai potensi besar untuk berkembang dan memacu pertumbuhan ekonomi. </s>
